<a href="https://colab.research.google.com/github/jeffgillan/Drone-Imagery-Analysis/blob/master/notebooks/STAC_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pystac_client. This library is used to crawl SpatioTemporal Asset Catalogs (STAC)
!pip install pystac_client --quiet
!pip install planetary_computer --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 4.9 MB/s eta 0:00:00


In [2]:
##Install library to work with raster images

! pip install rasterio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 21.7 MB/s eta 0:00:00


In [3]:
#Bring the libraries into the running environment

import pystac_client
import planetary_computer

In [4]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import requests
import numpy as np





In [6]:
from pystac_client import Client
catalog = Client.open('https://landsatlook.usgs.gov/stac-server/')
catalog.title

collections = list(catalog.get_collections())

# Print the number of collections
print(f"Number of collections in the base catalog: {len(collections)}")


Number of collections in the base catalog: 14


In [ ]:
# Print the names (or IDs) and descriptions of each collection
for collection in collections:
    print(f"ID: {collection.id}")
    print(f"Title: {collection.title}")  # Optional: if you want to print the title as well
    print(f"Description: {collection.description}")

In [26]:
#Do a temporal and geographic box search
time_range = "2020-12-01/2020-12-31"
bbox = [-111.185821, 32.047978, -110.690757, 32.41685]

search = catalog.search(collections=["landsat-c2ard-sr"], bbox=bbox, datetime=time_range)
items = search.item_collection()

# Print the total number of items found
print(f"Total number of items found: {len(items)}")

for item in items:
    print(f"Item ID: {item.id}")

    # Print the asset names (keys) for each item
    print("Assets:")
    for asset_key in item.assets:
        print(f"  - {asset_key}: {item.assets[asset_key].href}")
    print("---------------------------------------------------")

Total number of items found: 7
Item ID: LE07_CU_007014_20201231_20210504_02_SR
Assets:
  - thumbnail: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_thumb_small.jpeg
  - reduced_resolution_browse: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_thumb_large.jpeg
  - index: https://landsatlook.usgs.gov/stac-browser/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02
  - blue: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_SR_B1.TIF
  - green: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201231_20210504_02/LE07_CU_007014_20201231_20210504_02_SR_B2.TIF
  - red: https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE

In [19]:
import geopandas

In [20]:
df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

,geometry,start_datetime,end_datetime,datetime,platform,instruments,landsat:grid_horizontal,landsat:grid_vertical,landsat:grid_region,landsat:scene_count,eo:cloud_cover,landsat:cloud_shadow_cover,landsat:snow_ice_cover,landsat:fill,proj:epsg,proj:shape,proj:transform,proj:wkt2,created,updated
0,"POLYGON ((-112.06311 31.50502, -110.52665 31.7...",2020-12-31T17:16:20.3479967Z,2020-12-31T17:16:44.2400913Z,2020-12-31T17:16:32.294044Z,LANDSAT_7,[ETM],07,14,CU,2,66.3332,10.4581,0.0124,18.8851,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:46:19.451Z,2022-07-12T17:56:32.313Z
1,"POLYGON ((-110.98534 31.65586, -110.52636 31.7...",2020-12-24T17:10:38.1788617Z,2020-12-24T17:11:02.0712045Z,2020-12-24T17:10:50.125033Z,LANDSAT_7,[ETM],07,14,CU,2,100.0000,0.0000,0.0000,93.0905,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:45:26.992Z,2022-07-12T17:56:08.582Z
2,"POLYGON ((-112.09311 31.50066, -110.52636 31.7...",2020-12-23T17:58:09.1259540Z,2020-12-23T17:58:33.0169940Z,2020-12-23T17:58:21.071474Z,LANDSAT_8,"[OLI, TIRS]",07,14,CU,2,15.3390,8.3475,0.0011,3.5181,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:45:45.141Z,2022-07-12T17:46:32.781Z
3,"POLYGON ((-110.86078 31.67258, -110.52636 31.7...",NaN,NaN,2020-12-16T17:52:23.518400Z,LANDSAT_8,"[OLI, TIRS]",07,14,CU,1,0.0175,0.0522,0.0006,94.1125,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:45:23.209Z,2022-07-12T17:56:15.896Z
4,"POLYGON ((-112.09311 31.50066, -110.52685 31.7...",2020-12-15T17:17:29.3892952Z,2020-12-15T17:17:53.2809098Z,2020-12-15T17:17:41.335102Z,LANDSAT_7,[ETM],07,14,CU,2,0.0674,0.0700,0.0102,18.9629,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:45:23.061Z,2022-07-12T17:46:14.094Z
5,"POLYGON ((-111.00417 31.65332, -110.52636 31.7...",2020-12-08T17:11:50.3659923Z,2020-12-08T17:12:14.2571128Z,2020-12-08T17:12:02.311552Z,LANDSAT_7,[ETM],07,14,CU,2,0.1507,0.0618,0.0001,92.6755,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:45:18.915Z,2022-07-12T17:46:33.386Z
6,"POLYGON ((-112.09311 31.50066, -110.52636 31.7...",2020-12-07T17:58:11.1155250Z,2020-12-07T17:58:35.0065650Z,2020-12-07T17:58:23.061045Z,LANDSAT_8,"[OLI, TIRS]",07,14,CU,2,14.5915,2.2484,0.0000,3.4753,None,"[5000, 5000]","[30, 0, -1515585, 0, -30, 1214805]","PROJCS[""AEA WGS84"",GEOGCS[""WGS 84"",DATUM[""WGS_...",2022-07-07T13:16:54.996Z,2022-07-12T17:55:46.788Z


In [34]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
print(selected_item)

<Item id=LC08_CU_007014_20201216_20210504_02_SR>


In [32]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key                 ┃ Description                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ thumbnail                 │ Thumbnail image                                │
│ reduced_resolution_browse │ Reduced resolution browse image                │
│ index                     │ HTML index page                                │
│ blue                      │ Blue Band (B1)                                 │
│ green                     │ Green Band (B2)                                │
│ red                       │ Red Band (B3)                                  │
│ nir08                     │ Near Infrared Band 0.8 (B4)                    │
│ swir16                    │ Short-wave Infrared Band 1.6 (B5)              │
│ swir22                    │ Short-wave Infrared Band 2.2 (B7)              │
│ atmos_opacity             │ Atmospheric Opacity Band                       │
│ cloud_qa                  │ Cloud Quality Analysis Band                    │
│ qa_pixel                  │ Pixel Quality Assessment Band                  │
│ qa_radsat                 │ Radiometric Saturation Quality Assessment Band │
│ qa_lineage                │ Lineage Assessment Band                        │
└───────────────────────────┴────────────────────────────────────────────────┘

In [33]:
selected_item.assets["thumbnail"].to_dict()

{'href': 'https://landsatlook.usgs.gov/tile/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201224_20210504_02/LE07_CU_007014_20201224_20210504_02_thumb_small.jpeg',
 'type': 'image/jpeg',
 'title': 'Thumbnail image',
 'alternate': {'s3': {'storage:platform': 'AWS',
   'storage:requester_pays': True,
   'href': 's3://usgs-landsat-ard/collection02/etm/2020/CU/007/014/LE07_CU_007014_20201224_20210504_02/LE07_CU_007014_20201224_20210504_02_thumb_small.jpeg'}},
 'roles': ['thumbnail']}

In [35]:
from IPython.display import Image

Image(url=selected_item.assets["thumbnail"].href, width=500)

In [50]:
cog_path = 'https://landsatlook.usgs.gov/tile/collection02/oli-tirs/2020/CU/007/014/LC08_CU_007014_20201223_20210504_02/LC08_CU_007014_20201223_20210504_02_SR_B2.TIF'

In [51]:
save_path = '/content/STAC_test.tif'

In [52]:
response = requests.post(cog_path)

In [53]:
if response.status_code == 200:
    # Write the content of the response to a file in the specified save path
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully: {save_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

Failed to download the file. Status code: 403


In [41]:
image_path = '/content/STAC_blue.tif'

In [42]:
# Open the GeoTIFF file
with rasterio.open(image_path) as src:
    # Read the image data
    image = src.read(1)  # Reading the first band

    # Display the image
    plt.figure(figsize=(10, 10))
    show(image, transform=src.transform, cmap='gray')  # Use 'gray' colormap for single band images
    plt.title('GeoTIFF Single Band Image')
    plt.show()

RasterioIOError: '/content/STAC_blue.tif' not recognized as a supported file format.